# Training Portion
https://www.tensorflow.org/tutorials/images/classification

### Splitting of data into training and validation sets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
batch_size = 32
img_height = 100 #960
img_width = 350
#1350
data_dir = '/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/flipped'

In [ ]:
# Load the dataset and perform class-wise splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = dataset.class_names
num_classes = len(class_names)

# Create balanced validation and testing datasets
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 1021 files belonging to 15 classes.
Using 817 files for training.
Found 1021 files belonging to 15 classes.
Using 204 files for validation.


### Only needed if dataset is too huge, and need to train model with half of the dataset first

In [ ]:
# Calculate the number of samples in the training dataset
num_samples = dataset.cardinality().numpy()

# Split the dataset into two parts
split_index = num_samples // 2

# Create two separate datasets for the first and second parts
train_ds_part1 = dataset.take(split_index)
train_ds_part2 = dataset.skip(split_index)

### Preparing of Model

In [ ]:
# just to see if there are any extra folders
print(class_names)

['Barge', 'BulkCarrier', 'ContainerShip', 'Cruise', 'Dredger', 'Ferry', 'HarbourLaunch-PilotVessel', 'LNG-LPG', 'RORO', 'Sampan', 'SupplyVessel', 'Tanker-Bunker', 'Trawler-FishingVessel', 'Tugboat', 'Yacht']


In [ ]:
ship_categories = class_names

ship_dict = {index: category for index, category in enumerate(class_names)}

# print(ship_dict)


{0: 'Barge', 1: 'BulkCarrier', 2: 'ContainerShip', 3: 'Cruise', 4: 'Dredger', 5: 'Ferry', 6: 'HarbourLaunch-PilotVessel', 7: 'LNG-LPG', 8: 'RORO', 9: 'Sampan', 10: 'SupplyVessel', 11: 'Tanker-Bunker', 12: 'Trawler-FishingVessel', 13: 'Tugboat', 14: 'Yacht'}


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
model_1 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model_1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_7 (Rescaling)     (None, 100, 350, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 100, 350, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 50, 175, 16)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 50, 175, 32)       4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 25, 87, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 25, 87, 64)       

### For transfer learning purposes
(training existing models)

In [ ]:
from tensorflow.keras.models import load_model
model_1 = load_model('/content/drive/MyDrive/flir_model_withSharpenedBrightened.keras')

### Model Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
epochs=50

# EarlyStopping will stop training if validation loss doesn't improve
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train your model
history = model_1.fit(
    dataset,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)

# Find the index where validation loss is minimized
best_epoch = history.history['val_loss'].index(min(history.history['val_loss']))

# Number of epochs trained before early stopping
epochs_trained = best_epoch + 1
print("Number of epochs trained:", epochs_trained)

Epoch 1/50
26/26 [==============================] - 9s 82ms/step - loss: 3.0500 - accuracy: 0.3060 - val_loss: 2.0685 - val_accuracy: 0.4118
Epoch 2/50
26/26 [==============================] - 1s 39ms/step - loss: 1.9529 - accuracy: 0.4186 - val_loss: 2.0553 - val_accuracy: 0.4118
Epoch 3/50
26/26 [==============================] - 1s 30ms/step - loss: 1.8568 - accuracy: 0.4247 - val_loss: 2.0391 - val_accuracy: 0.4118
Epoch 4/50
26/26 [==============================] - 1s 26ms/step - loss: 1.6807 - accuracy: 0.4737 - val_loss: 1.8338 - val_accuracy: 0.4559
Epoch 5/50
26/26 [==============================] - 1s 31ms/step - loss: 1.4628 - accuracy: 0.5275 - val_loss: 1.7963 - val_accuracy: 0.4363
Epoch 6/50
26/26 [==============================] - 1s 42ms/step - loss: 1.3078 - accuracy: 0.5520 - val_loss: 1.7379 - val_accuracy: 0.4657
Epoch 7/50
26/26 [==============================] - 1s 32ms/step - loss: 1.0131 - accuracy: 0.6536 - val_loss: 2.0240 - val_accuracy: 0.4559
Epoch 8/50
26

#### Use only if need to train model 2 times (when dataset too huge)

##### Part 1

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
epochs=50

# EarlyStopping will stop training if validation loss doesn't improve
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

checkpoint = ModelCheckpoint('/content/drive/MyDrive/2023_internship/dataset_20200803/model0_phase1.h5', save_best_only=True)

# Train your model
history = model_1.fit(
    train_ds_part1,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping, checkpoint]
)

# Find the index where validation loss is minimized
best_epoch = history.history['val_loss'].index(min(history.history['val_loss']))

# Number of epochs trained before early stopping
epochs_trained = best_epoch + 1
print("Number of epochs trained:", epochs_trained)

Epoch 1/50
267/267 [==============================] - 1461s 5s/step - loss: 1.6635 - accuracy: 0.4953 - val_loss: 1.2465 - val_accuracy: 0.6224
Epoch 2/50
267/267 [==============================] - 42s 154ms/step - loss: 0.8183 - accuracy: 0.7511 - val_loss: 0.7375 - val_accuracy: 0.7870
Epoch 3/50
267/267 [==============================] - 41s 151ms/step - loss: 0.3407 - accuracy: 0.8984 - val_loss: 0.6164 - val_accuracy: 0.8390
Epoch 4/50
267/267 [==============================] - 41s 150ms/step - loss: 0.1328 - accuracy: 0.9630 - val_loss: 0.5093 - val_accuracy: 0.8823
Epoch 5/50
267/267 [==============================] - 41s 149ms/step - loss: 0.0727 - accuracy: 0.9800 - val_loss: 0.5235 - val_accuracy: 0.9096
Epoch 6/50
267/267 [==============================] - 40s 148ms/step - loss: 0.0570 - accuracy: 0.9833 - val_loss: 0.5826 - val_accuracy: 0.9190
Epoch 7/50
267/267 [==============================] - 39s 145ms/step - loss: 0.0590 - accuracy: 0.9843 - val_loss: 0.6766 - val_acc

##### Part 2

In [ ]:
# Load the saved model's weights from the first phase
model_1.load_weights('/content/drive/MyDrive/2023_internship/dataset_20200803/model0_phase1.h5')

checkpoint = ModelCheckpoint('/content/drive/MyDrive/2023_internship/dataset_20200803/model0_phase2.h5', save_best_only=True)

# Continue training the model on the remaining part of the dataset
history_phase2 = model_1.fit(
    train_ds_part2,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping, checkpoint]
)

# Find the index where validation loss is minimized
best_epoch = history.history['val_loss'].index(min(history.history['val_loss']))

# Number of epochs trained before early stopping
epochs_trained = best_epoch + 1
print("Number of epochs trained:", epochs_trained)

Epoch 1/50
267/267 [==============================] - 910s 3s/step - loss: 0.2866 - accuracy: 0.9273 - val_loss: 0.1481 - val_accuracy: 0.9581
Epoch 2/50
267/267 [==============================] - 72s 145ms/step - loss: 0.0465 - accuracy: 0.9882 - val_loss: 0.0890 - val_accuracy: 0.9721
Epoch 3/50
267/267 [==============================] - 72s 142ms/step - loss: 0.0310 - accuracy: 0.9924 - val_loss: 0.0942 - val_accuracy: 0.9787
Epoch 4/50
267/267 [==============================] - 72s 145ms/step - loss: 0.0698 - accuracy: 0.9785 - val_loss: 0.1612 - val_accuracy: 0.9455
Epoch 5/50
267/267 [==============================] - 75s 157ms/step - loss: 0.0515 - accuracy: 0.9861 - val_loss: 0.0877 - val_accuracy: 0.9717
Epoch 6/50
267/267 [==============================] - 73s 144ms/step - loss: 0.0113 - accuracy: 0.9973 - val_loss: 0.0481 - val_accuracy: 0.9860
Epoch 7/50
267/267 [==============================] - 72s 144ms/step - loss: 0.0088 - accuracy: 0.9984 - val_loss: 0.0626 - val_accu

### Save the Model

In [ ]:
model_1.save('/content/drive/MyDrive/flirModel_withSharpenedBrightened_onlyFlipped.keras')